# NOTE: Make sure you choose Runtime type = 'GPU'

## Also, have a look at the Functional API

In [0]:
# Upload the dogscats.zip file.

from google.colab import files
uploaded = files.upload()

Saving dogscats.zip to dogscats.zip


In [0]:
!unzip dogscats.zip

In [0]:
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Flatten, Activation
from keras import backend as K
from keras.applications.vgg16 import VGG16

In [0]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(200,200,3))
print('Model loaded.')

Model loaded.


In [0]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator()

In [0]:
# add a global spatial average pooling layer
x = base_model.output

x = Conv2D(2,1,1)(x)
x = GlobalAveragePooling2D()(x)
predictions = Activation('softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0         
__________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(2, (1, 1))`
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
# Making all the layers non-trainable.
for layer in base_model.layers:
    layer.trainable = False

In [0]:
train_data_dir='dogscats/train'
test_data_dir='dogscats/valid'
img_height=200
img_width=200
batch_size=32

In [0]:
# Training generator

train_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
  ) # set as training data

Found 200 images belonging to 2 classes.


In [0]:
# Testing Generator

test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(200, 200),
        batch_size=32,
        class_mode='categorical')

Found 100 images belonging to 2 classes.


In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
epochs=10

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = test_generator, 
    validation_steps = test_generator.samples // batch_size,
    epochs = epochs)

Epoch 1/10
6/6 [==============================] - 2s 368ms/step - loss: 0.0960 - acc: 0.9628 - val_loss: 0.2018 - val_acc: 0.9706
Epoch 2/10
6/6 [==============================] - 2s 390ms/step - loss: 0.0649 - acc: 0.9792 - val_loss: 0.2319 - val_acc: 0.9375
Epoch 3/10
6/6 [==============================] - 2s 337ms/step - loss: 0.0670 - acc: 0.9600 - val_loss: 0.1760 - val_acc: 0.9412
Epoch 4/10
6/6 [==============================] - 2s 335ms/step - loss: 0.0417 - acc: 0.9787 - val_loss: 0.2364 - val_acc: 0.9559
Epoch 5/10
6/6 [==============================] - 2s 332ms/step - loss: 0.0200 - acc: 0.9947 - val_loss: 0.1575 - val_acc: 0.9559
Epoch 6/10
6/6 [==============================] - 2s 365ms/step - loss: 0.0176 - acc: 1.0000 - val_loss: 0.2074 - val_acc: 0.9479
Epoch 7/10
6/6 [==============================] - 2s 333ms/step - loss: 0.0220 - acc: 1.0000 - val_loss: 0.1526 - val_acc: 0.9706
Epoch 8/10
6/6 [==============================] - 2s 330ms/step - loss: 0.0135 - acc: 1.00